# Wine Quality Notebook
The data used in this notebook is a variation of the wine quality dataset.

This dataset is public available for research. The details are described in [Cortez et al., 2009]. 
  Please include this citation if you plan to use this database:

  P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
  Modeling wine preferences by data mining from physicochemical properties.
  In Decision Support Systems>, Elsevier, 47(4):547-553. ISSN: 0167-9236.

  Available at: [@Elsevier] http://dx.doi.org/10.1016/j.dss.2009.05.016
                [Pre-press (pdf)] http://www3.dsi.uminho.pt/pcortez/winequality09.pdf
                [bib] http://www3.dsi.uminho.pt/pcortez/dss09.bib

1. Title: Wine Quality 

2. Sources
   Created by: Paulo Cortez (Univ. Minho), António Cerdeira, Fernando Almeida, Telmo Matos and José Reis (CVRVV) @ 2009
   
3. Past Usage:

  P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
  Modeling wine preferences by data mining from physicochemical properties.
  In Decision Support Systems>, Elsevier, 47(4):547-553. ISSN: 0167-9236.

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers

2024-05-21 19:04:50.072127: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 19:04:50.072189: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 19:04:50.072226: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 19:04:50.083144: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
GOOGLE_CLOUD_PROJECT = 'par-devfest-sfeir'
GOOGLE_CLOUD_DATASET = 'wine_quality'
RED_WINE_TABLE = 'red_wine'
WHITE_WINE_TABLE = 'white_wine'

Construct a BigQuery Client agent to interact with BigQuery

In [3]:
from google.cloud import bigquery
client = bigquery.Client(project=GOOGLE_CLOUD_PROJECT)

In [4]:
sql_query = f"""
SELECT *
FROM `{GOOGLE_CLOUD_PROJECT}.{GOOGLE_CLOUD_DATASET}.{WHITE_WINE_TABLE}`
"""
white_wine_df = client.query(sql_query).to_dataframe()

Split the data into training and validation datasets

In [5]:
df_train = white_wine_df.sample(frac=0.8, random_state=42)
df_valid = white_wine_df.drop(df_train.index)
display(df_train.head(10))

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality
4656,6.9,0.190,0.38,1.15,0.023,30.0,105.0,0.99047,3.11,0.38,11.4,0
3659,6.2,0.170,0.28,4.70,0.037,39.0,133.0,0.99310,3.41,0.46,10.8,1
907,7.8,0.320,0.31,1.70,0.036,46.0,195.0,0.99300,3.03,0.48,10.5,0
4352,6.6,0.300,0.74,4.60,0.041,36.0,159.0,0.99460,3.21,0.45,9.9,0
3271,6.2,0.320,0.12,4.80,0.054,6.0,97.0,0.99424,3.16,0.50,9.3,0
4632,7.4,0.220,0.33,2.00,0.045,31.0,101.0,0.99310,3.42,0.55,11.4,0
2244,7.2,0.550,0.09,1.50,0.108,16.0,151.0,0.99380,3.07,0.57,9.2,0
1924,7.2,0.280,0.38,2.00,0.052,23.0,156.0,0.99120,3.13,0.52,11.1,0
3801,6.4,0.230,0.32,1.90,0.038,40.0,118.0,0.99074,3.32,0.53,11.8,1
2634,5.6,0.225,0.24,9.80,0.054,59.0,140.0,0.99545,3.17,0.39,10.2,1


Normalize the data

In [6]:
X_train = df_train.drop('quality', axis=1)
X_valid = df_valid.drop('quality', axis=1)

X_train = (X_train - X_train.min())/(X_train.max() - X_train.min())
X_valid = (X_valid - X_valid.min())/(X_valid.max() - X_valid.min())

y_train = df_train["quality"]
y_valid = df_valid["quality"]

y_train = np.asarray(y_train).astype(np.int_)
y_valid = np.asarray(y_valid).astype(np.int_)

In [7]:
X_train.head(10)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol
4656,0.379747,0.107843,0.228916,0.008436,0.041543,0.097561,0.220930,0.064777,0.357798,0.186047,0.548387
3659,0.291139,0.088235,0.168675,0.062883,0.083086,0.128920,0.286047,0.115481,0.633028,0.279070,0.451613
907,0.493671,0.235294,0.186747,0.016871,0.080119,0.153310,0.430233,0.113553,0.284404,0.302326,0.403226
4352,0.341772,0.215686,0.445783,0.061350,0.094955,0.118467,0.346512,0.144399,0.449541,0.267442,0.306452
3271,0.291139,0.235294,0.072289,0.064417,0.133531,0.013937,0.202326,0.137459,0.403670,0.325581,0.209677
4632,0.443038,0.137255,0.198795,0.021472,0.106825,0.101045,0.211628,0.115481,0.642202,0.383721,0.548387
2244,0.417722,0.460784,0.054217,0.013804,0.293769,0.048780,0.327907,0.128976,0.321101,0.406977,0.193548
1924,0.417722,0.196078,0.228916,0.021472,0.127596,0.073171,0.339535,0.078851,0.376147,0.348837,0.500000
3801,0.316456,0.147059,0.192771,0.019939,0.086053,0.132404,0.251163,0.069983,0.550459,0.360465,0.612903
2634,0.215190,0.142157,0.144578,0.141104,0.133531,0.198606,0.302326,0.160787,0.412844,0.197674,0.354839


In [8]:
model = keras.Sequential([
    layers.Dense(units=16, activation='relu', input_shape=[11]),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(units=32, activation='relu'),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(units=32, activation='relu'),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(units=1, activation=tf.keras.activations.sigmoid)
])

In [9]:
rms_prop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=10e-3)
model.compile(
      optimizer=rms_prop_optimizer,
      loss=tf.keras.losses.binary_crossentropy,
      metrics=['accuracy']
  )

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', patience=10)

history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=64,
    epochs=200,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/200
62/62 [==============================] - 2s 7ms/step - loss: 0.5857 - accuracy: 0.7006 - val_loss: 0.6141 - val_accuracy: 0.6857
Epoch 2/200
62/62 [==============================] - 0s 4ms/step - loss: 0.5379 - accuracy: 0.7279 - val_loss: 0.5763 - val_accuracy: 0.6857
Epoch 3/200
62/62 [==============================] - 0s 4ms/step - loss: 0.5307 - accuracy: 0.7312 - val_loss: 0.5430 - val_accuracy: 0.6857
Epoch 4/200
62/62 [==============================] - 0s 4ms/step - loss: 0.5308 - accuracy: 0.7338 - val_loss: 0.5410 - val_accuracy: 0.6939
Epoch 5/200
62/62 [==============================] - 0s 3ms/step - loss: 0.5259 - accuracy: 0.7381 - val_loss: 0.5190 - val_accuracy: 0.7602
Epoch 6/200
62/62 [==============================] - 0s 4ms/step - loss: 0.5156 - accuracy: 0.7414 - val_loss: 0.5050 - val_accuracy: 0.7653
Epoch 7/200
62/62 [==============================] - 0s 3ms/step - loss: 0.5114 - accuracy: 0.7473 - val_loss: 0.5022 - val_accuracy: 0.7520
Epoch 8/200
6